# Advent of Code 2025

### Common code

In [ ]:
import re

from typing import Literal, Iterable

def read_input_lines(*, day: int) -> Iterable[str]:
    """
    Reads an input file for a given day and difficulty, yielding one line at a time.
    """
    with open(f"input/{day:02}", 'r') as file:
        yield from file.readlines()

def parse_input(pattern: re.Pattern, *, day: int) -> Iterable[re.Match]:
    """
    Parse an input file line-by-line, yielding the given regex pattern matches.
    """
    for line in read_input_lines(day=day):
        yield pattern.match(line)

def natural_mod(x: int, m: int) -> int:
    """
    "Natural numbers" modulo, as such that the modulo of negative numbers will instead loop back to positive.
    """
    return (x % m + m) % m


## Day 1 - Secret entrance

In [ ]:
input_pattern = re.compile(r"(?P<direction>L|R)(?P<value>\d+)")

DIAL_START_POS: int = 50
DIAL_SIZE: int = 100

In [ ]:
def day_1_easy() -> int:

    current_pos = DIAL_START_POS
    zero_passes = 0

    for line in parse_input(input_pattern, day=1):
        (direction, value) = line.groupdict().values()
        value = int(value)
        
        match direction:
            case 'L': current_pos = natural_mod(current_pos - value, DIAL_SIZE)
            case 'R': current_pos = natural_mod(current_pos + value, DIAL_SIZE)
            case _:
                return ValueError(f"'{direction}' is not a valid direction")
                
        if current_pos == 0:
            zero_passes += 1

    return zero_passes

day_1_easy()

In [ ]:
def day_1_hard() -> int:

    class Dial:

        current_pos: int
        zero_passes: int

        def __init__(self):
            self.current_pos = DIAL_START_POS
            self.zero_passes = 0

        def move(self, value: int, clockwise: bool) -> None:
            click = +1 if clockwise else -1
            for _ in range(value):
                self.current_pos += click

                if self.current_pos == DIAL_SIZE:
                    self.current_pos = 0
                    
                elif self.current_pos == -1:
                    self.current_pos = DIAL_SIZE - 1
                
                if self.current_pos == 0:
                    self.zero_passes += 1

    dial = Dial()

    for line in parse_input(input_pattern, day=1):
        (direction, value) = line.groupdict().values()
        value = int(value)

        match direction:
            case 'L': dial.move(value, False)
            case 'R': dial.move(value, True)
            case _:
                return ValueError(f"'{direction}' is not a valid direction")

    return dial.zero_passes

day_1_hard()

## Day 2 - Gift Shop

In [ ]:
from math import ceil

def parse_ranges() -> list[range]:
    line: str = next(read_input_lines(day=2))
    for range_ in line.split(","):
        (start, stop) = range_.split("-")
        yield range(int(start), int(stop) + 1)

def get_nlength_number(length: int) -> int:
    """
    Returns the smallest number of length `length`, as a str.
    """
    assert length > 0, "Length must be at least 1"
    return 10 ** (length - 1)

type NormalRange = range # Simple label to differentiate "normalized" ranges
def normalize(ranges: list[range]) -> list[NormalRange]:
    """
    Yield from ranges, splitting a range if its start and stop differ in number of digits,
    such that all ranges in the resulting collection have a start and stop of the same 10th power order.
    """
    for range_ in ranges:
        start_len = len(str(range_.start))
        stop_len = len(str(range_.stop - 1))

        start = range_.start
        sep = start
        for i in range(start_len, stop_len):
            sep = get_nlength_number(i + 1)
            yield range(start, sep)
            start = sep

        yield range(sep, range_.stop)

In [ ]:
def day_2_easy() -> int:

    total: int = 0

    for range_ in normalize(parse_ranges()):
        number_length = len(str(range_.start))

        if number_length % 2 != 0:
            continue
        pattern_length = number_length // 2

        start_half = int(str(range_.start)[:pattern_length])
        stop_half = int(str(range_.stop - 1)[:pattern_length])
        for pattern in range(start_half, stop_half + 1):
            if (result := int(str(pattern) * 2)) in range_:
                total += result

    return total

day_2_easy()

In [ ]:
def day_2_hard() -> int:

    def find_patterned_numbers(range_: NormalRange) -> set[int]:
        """
        For a given "normal range", find all "patterned" numbers.
        """
        found = set[int]()
        
        number_length = len(str(range_.start))
        for pattern_length in range(1, number_length // 2 + 1):
            if number_length % pattern_length == 0:
                repeat = number_length // pattern_length
                found.update(
                    result
                    for pattern in range(
                        int(str(range_.start)[:pattern_length]),   # start
                        int(str(range_.stop - 1)[:pattern_length]) + 1 # stop
                    )
                    if (result := int(str(pattern) * repeat)) in range_
                )
                
        return found
    
    return sum(
        sum(find_patterned_numbers(range_))
        for range_ in normalize(parse_ranges())
    )

day_2_hard()

## Day 3 - Lobby

In [ ]:
type Bank = list[int]
line_pattern = re.compile(r"(\d+)")

def parse_banks() -> list[Bank]:
    for match in parse_input(line_pattern, day=3):
        yield [int(digit) for digit in match.group(1)]

In [ ]:
def day_3_easy() -> int:

    total_joltage: int = 0

    for bank in parse_banks():

        first_digit_i = bank.index(max(bank[:-1]))

        second_bank = bank[first_digit_i + 1:]
        second_digit_i = first_digit_i + 1 + second_bank.index(max(second_bank))

        total_joltage += 10 * bank[first_digit_i] + bank[second_digit_i]

    return total_joltage

day_3_easy()

In [ ]:
BATTERIES_COUNT: int = 12

def day_3_hard() -> int:

    total_joltage: int = 0

    for bank in parse_banks():

        selected_batteries = list(range(
            len(bank) - BATTERIES_COUNT,
            len(bank)
        ))

        lower_bound = -1
        for i in range(BATTERIES_COUNT):
            for j in range(selected_batteries[i] - 1, lower_bound, -1):
                
                if bank[j] >= bank[selected_batteries[i]]:
                    selected_batteries[i] = j
                    
            lower_bound = selected_batteries[i]

        value = "".join(str(bank[i]) for i in selected_batteries)
        total_joltage += int(value)

    return total_joltage

day_3_hard()

## Day 4 - Printing department

In [ ]:
from dataclasses import dataclass, field
from typing import Self

type Coord = tuple[int, int]

@dataclass
class Grid:
    height: int
    width: int
    rolls: set[Coords] = field(default_factory=set)

    @staticmethod
    def read_input() -> Self:
        """
        Reads a grid from `read_input_lines`.
        """
        rolls = set[Coord]()
        for (i, line) in enumerate(read_input_lines(day=4)):
            for (j, cell) in enumerate(line):
                if cell == "@":
                    rolls.add((i, j))

        g = Grid(i + 1, j + 1)
        g.rolls.update(rolls)
        return g

    def cluttering_at(self, pos: Coord) -> int:
        (i, j) = pos
        return sum(
            1
            for a in range(max(0, i - 1), min(i + 2, self.height))
            for b in range(max(0, j - 1), min(j + 2, self.width))
            if pos in self.rolls and (a, b) in self.rolls and (a, b) != pos
        )

MAX_CLUTTER: int = 4


In [ ]:
def day_4_easy() -> int:

    result: int = 0

    grid = Grid.read_input()
    for roll in grid.rolls:
        
        if grid.cluttering_at(roll) < MAX_CLUTTER:
            result += 1
            
    return result    

day_4_easy()

In [ ]:
def day_4_hard() -> int:
    
    result: int = 0

    grid = Grid.read_input()
    next_rolls = grid.rolls.copy

    while True:
        next_rolls = grid.rolls.copy()

        for roll in grid.rolls:

            if grid.cluttering_at(roll) < MAX_CLUTTER:
                next_rolls.remove(roll)
                result += 1

        if next_rolls != grid.rolls:
            grid.rolls = next_rolls.copy()
        else:
            break

    return result

day_4_hard()

## Day 5 - Cafeteria

In [ ]:
RANGE_PATTERN = re.compile(r"(?P<start>\d+)-(?P<stop>\d+)")

def parse_ingredients() -> tuple[list[range], list[int]]:
    lines = list(read_input_lines(day=5))
    split = lines.index("\n")
    (ranges, ids) = (lines[:split], lines[split + 1:])
    return (
        # Parse ranges
        [
            range(
                int(match.group("start")),
                int(match.group("stop")) + 1
            )
            for match in (
                RANGE_PATTERN.match(line)
                for line in ranges
            )
        ],
        # Parse IDs
        [
            int(line) for line in ids
        ]
    )

In [ ]:
def day_5_easy() -> int:
    (ranges, ids) = parse_ingredients()

    return sum(
        1 for i in ids
        if any(i in range_ for range_ in ranges)
    )

day_5_easy()

In [ ]:
def day_5_hard() -> int:
    (ranges, _) = parse_ingredients()
    ranges.sort(key = lambda r: r.start)

    reduced_ranges = list[range]()
    current_range = ranges[0]

    for range_ in ranges[1:]:

        if range_.start < current_range.stop:
            current_range = range(
                current_range.start,
                max(current_range.stop, range_.stop)
            )

        else:
            reduced_ranges.append(current_range)
            current_range = range_

    reduced_ranges.append(current_range)

    return sum(range_.stop - range_.start for range_ in reduced_ranges)

day_5_hard()        

## Day 6 - Trash compactor

In [ ]:
from typing import Callable

OPERATORS: dict[str, Callable[[int, int], int]] = {
    "+": lambda a, b: a + b,
    "*": lambda a, b: a * b
}

In [ ]:
from functools import reduce

def day_6_easy() -> int:
    # Read the entire input at once, to separate the last row from the rest
    lines = list(read_input_lines(day=6))

    # Transpose the input "grid", minus the last line (treat columns instead of rows)
    operand_groups = zip(*((int(number) for number in line.split()) for line in lines[:-1]))
    operators = [OPERATORS[symbol] for symbol in lines[-1].split()]

    return sum(
        reduce(operator, operands)
        for (operands, operator) in zip(operand_groups, operators)
    )

day_6_easy()

In [ ]:
from functools import reduce

def day_6_hard() -> int:
    lines = list(read_input_lines(day=6))

    # Zip'n'reverse sorcery to morph the input into the right-to-left vertical reading direction
    operands = ["".join(chars) for chars in zip(*(reversed(list(line)) for line in lines[:-1]))][1:]
    
    operators = iter(reversed([OPERATORS[symbol] for symbol in lines[-1].split()]))

    total: int = 0
    buffer = list[int]()
    for operand in operands: 

        if operand.isspace(): # ' ' empty str marks end of group
            total += reduce(next(operators), buffer)
            buffer.clear()

        else:
            buffer.append(int(operand))

    # Treat remaining buffer
    total += reduce(next(operators), buffer)
    return total
    
day_6_hard()